In [13]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.layers import Dense, Embedding, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from tensorflow.keras.preprocessing.sequence import pad_sequences





In [5]:
df = pd.read_csv('Emotion_final.csv')

In [6]:
labels = sorted(list(set(df.Emotion)))
for i in range(len(labels)):
    df.Emotion[df.Emotion == labels[i]] = i

In [7]:
labels_map = {i: labels[i] for i in range(len(labels))}
labels_map

{0: 'anger', 1: 'fear', 2: 'happy', 3: 'love', 4: 'sadness', 5: 'surprise'}

In [8]:
X = df['Text']
y = df['Emotion']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [9]:
y_train = np_utils.to_categorical(y_train, len(labels))
y_test = np_utils.to_categorical(y_test, len(labels))

In [10]:
X_train[:5]

9586     i had not yet gotten married and that coupled ...
5847     im feeling cautiously optimistic about the dir...
1681     i felt such a resonance with your words i feel...
1452     i like this so much but i feel like somehow th...
15890    i don t feel particularly passionate as i once...
Name: Text, dtype: object

In [12]:
tokenizer = Tokenizer(num_words=10000 )
tokenizer.fit_on_texts(X_train)

In [14]:
x_train = tokenizer.texts_to_sequences(X_train)
x_train = pad_sequences(x_train, maxlen=30)

In [19]:
model = Sequential()
model.add(Embedding(10000, 32, input_length=30))
model.add(Conv1D(250, 5, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(len(labels), activation='softmax'))

In [20]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [21]:
model.fit(x_train, y_train, batch_size=32, epochs=20)

Epoch 1/20
450/450 [==============================] - 5s 8ms/step - loss: 1.0814 - accuracy: 0.5763
Epoch 2/20
450/450 [==============================] - 4s 8ms/step - loss: 0.3143 - accuracy: 0.8896
Epoch 3/20
450/450 [==============================] - 3s 7ms/step - loss: 0.1540 - accuracy: 0.9451
Epoch 4/20
450/450 [==============================] - 3s 7ms/step - loss: 0.0769 - accuracy: 0.9713
Epoch 5/20
450/450 [==============================] - 3s 7ms/step - loss: 0.0441 - accuracy: 0.9847
Epoch 6/20
450/450 [==============================] - 3s 7ms/step - loss: 0.0275 - accuracy: 0.9910
Epoch 7/20
450/450 [==============================] - 3s 7ms/step - loss: 0.0202 - accuracy: 0.9938
Epoch 8/20
450/450 [==============================] - 4s 8ms/step - loss: 0.0189 - accuracy: 0.9939
Epoch 9/20
450/450 [==============================] - 3s 7ms/step - loss: 0.0141 - accuracy: 0.9963
Epoch 10/20
450/450 [==============================] - 3s 8ms/step - loss: 0.0139 - accuracy: 0.9957

In [22]:
x_test = tokenizer.texts_to_sequences(X_test)
x_test = pad_sequences(x_test, maxlen=30)

In [23]:
scores = model.evaluate(x_test, y_test)
print(f'Accuracy: {scores[1]}')

222/222 [==============================] - 1s 4ms/step - loss: 0.8328 - accuracy: 0.8750
Accuracy: 0.875035285949707
